In [1]:
import pickle
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score, average_precision_score
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:

perplexions_path = '../data/unified_clean_ant_pairs_formated_def.pickle'
test_dataset_path = '../data/ant_test.txt'

with open(perplexions_path, 'rb') as f: #(ребенок, родитель): перплексия
    ppls = pickle.load(f)


ppls_clean = dict()
for item in ppls.items():
    ppls_clean[(item[0][0].split('(')[0].strip(', '), item[0][1].strip(', '))] = item[1]


In [7]:
pairs = []
y_true = []
non_reversed = []
not_found = []
data = []

with open(test_dataset_path, 'r',encoding='utf-8') as f:
    i = 0
    lines = f.readlines()
    for line in lines:
        ex1, ex2, category = line.strip('\n').split('\t')
        s11, v1, s12 = ex1.split(',')
        s21, v2, s22 = ex2.split(',')
        # if s11 == s21 and s12 == s22:
        v1 = v1.strip(' ')
        v2 = v2.strip(' ')
        if category == 'directional_entailment': # child, parent
            data.append((v1, v2, 1))

        elif category == 'directional_non-entailment': # parent, child
            data.append((v1, v2, 0))
        # else:
        #     non_reversed.append((s11, s12, v1, s21, s22, v2, category))

y_true = [elem[2] for elem in data]

In [34]:
df = {'forward':[], 'backward':[], 'target':[]}

for child, parent, label in data:
    if not (child, parent) in ppls_clean.keys():
        #y_pred.append(1)
        continue
    
    
    forward_ppl = ppls_clean[(child, parent)]
    backward_ppl = ppls_clean[(parent, child)]
    df['forward'].append(forward_ppl)
    df['backward'].append(backward_ppl)
    df['target'].append(label)

In [35]:
X = pd.DataFrame(df)

In [36]:
X['frac'] = X['backward'] / X['forward']
X['diff'] = X['backward'] - X['forward']
X['mul'] = X['backward'] * X['forward']
X['sign'] = X['frac'] < 0.5

scaler = preprocessing.Normalizer()
X[['forward', 'backward', 'frac', 'diff', 'mul']] = scaler.fit_transform(X[['forward', 'backward', 'frac', 'diff', 'mul']] )

In [38]:
dropping = ['target', 'mul', 'sign', 'frac', 'backward', 'frac']

logreg = LogisticRegression(C=1, max_iter=1000, fit_intercept=False)
logreg.fit(X.drop(columns=dropping), X['target'])

probas = logreg.predict_proba(X.drop(columns=dropping))
y_pred = probas[:,1]
y_true = X['target']

roc_auc =  roc_auc_score(y_true, y_pred)
ap = average_precision_score(y_true, y_pred)

roc_auc, ap

(0.49965350460786445, 0.5077852708148656)

In [39]:
logreg.coef_, logreg.intercept_

(array([[1.17111725]]), array([0.]))